# Text Data Processing

In this assignment we are writing the following 2 functions for Text Data Processing

<ul>
    <li>
        <b>preprocess:</b> Function takes in a pandas.Series() of a corpus of text data as an argument. This function should output an indexed vocabulary and preprocessed tokens.
    </li>
    <li>
        <b>encode():</b> Function that takes in two arguments: 1) a pandas.Series() (or the preprocessed token outputs of the preprocess() function), and 2) a specified encoding method. These encoding methods must include Bag-of-Words, TF-IDF, and Word2Vec. 
    </li>
</ul>

In [1]:
!pip install nltk
!pip install contractions
!pip install inflect
!pip install scikit-learn 
!pip install gensim

DEPRECATION: uvicorn 0.14.0 has a non-standard dependency specifier click>=7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of uvicorn or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: uvicorn 0.14.0 has a non-standard dependency specifier click>=7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of uvicorn or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: uvicorn 0.14.0 has a non-standard dependency specifier click>=7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of uvicorn or contact the author to suggest that they release a version with a conforming depen

In [2]:
import pandas as pd
import numpy as np
import sklearn
from IPython.display import display, HTML

# Display Properties
from IPython.display import display, HTML
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
import nltk
import string
import re
import inflect
import contractions
from data_pipeline import Text_Pipeline

# Download the various 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Initialize various tools
text_pipeline = Text_Pipeline('CONVERT')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
import pandas as pd 

CORPUS = [
    "The quick brown fox jumps over the lazy dog",
    "A king's strength also includes his allies",
    "History is written by the victors",
    "An apple a day keeps the doctor away",
    "Nothing happens until something moves",
    "The 10,000,303 striped bats    aren't hanging on their feet for best."
    ]

# Create a Pandas series 
s = pd.Series(CORPUS) 

# Obtain pre processed series
preprocessed_series = text_pipeline.preprocess(s)
print(preprocessed_series)

0                        quick brown fox jump lazy dog
1                      king strength also include ally
2                                 history write victor
3                           apple day keep doctor away
4                       nothing happens something move
5    ten million three hundred three strip bat hang...
dtype: object


In [5]:
import pandas as pd

# Get matrix using BOW
matrix, column_names = text_pipeline.encode(preprocessed_series, 'BOW')

result = pd.DataFrame(
    data=matrix.toarray(), 
    index=preprocessed_series.values, 
    columns=column_names
)

result.head()

,ally,also,apple,away,bat,best,brown,day,doctor,dog,foot,fox,hang,happens,history,hundred,include,jump,keep,king,lazy,million,move,nothing,quick,something,strength,strip,ten,three,victor,write
quick brown fox jump lazy dog,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0
king strength also include ally,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
history write victor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
apple day keep doctor away,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
nothing happens something move,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0


In [6]:
# Get matrix using TF-IDF
matrix, column_names = text_pipeline.encode(preprocessed_series, 'TFIDF')

result = pd.DataFrame(
    data=matrix.toarray(), 
    index=preprocessed_series.values, 
    columns=column_names
)

result.head()

,ally,also,apple,away,bat,best,brown,day,doctor,dog,foot,fox,hang,happens,history,hundred,include,jump,keep,king,lazy,million,move,nothing,quick,something,strength,strip,ten,three,victor,write
quick brown fox jump lazy dog,0.000,0.000,0.000,0.000,0.000,0.000,0.408,0.000,0.000,0.408,0.000,0.408,0.000,0.000,0.000,0.000,0.000,0.408,0.000,0.000,0.408,0.000,0.000,0.000,0.408,0.000,0.000,0.000,0.000,0.000,0.000,0.000
king strength also include ally,0.447,0.447,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.447,0.000,0.000,0.447,0.000,0.000,0.000,0.000,0.000,0.000,0.447,0.000,0.000,0.000,0.000,0.000
history write victor,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.577,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.577,0.577
apple day keep doctor away,0.000,0.000,0.447,0.447,0.000,0.000,0.000,0.447,0.447,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.447,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
nothing happens something move,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.500,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.500,0.500,0.000,0.500,0.000,0.000,0.000,0.000,0.000,0.000


In [7]:
# Get matrix using Word to Vector
matrix = text_pipeline.encode(preprocessed_series, 'WordToVec')

result = pd.DataFrame(
    data=matrix.vectors, 
    index=matrix.key_to_index.keys()
)

result.head()

,0,1,2,3,4,5,6,7,8,9
three,-0.005,0.002,0.051,0.090,-0.093,-0.071,0.065,0.090,-0.050,-0.038
best,0.074,-0.015,-0.045,0.066,-0.049,-0.018,0.029,0.010,-0.083,-0.095
apple,0.073,0.051,0.068,0.008,0.064,-0.034,-0.009,0.058,-0.075,-0.039
brown,-0.075,-0.009,0.095,-0.073,-0.023,-0.019,0.081,-0.059,0.000,-0.048
fox,-0.096,0.050,-0.088,-0.044,-0.000,-0.003,-0.077,0.096,0.050,0.092
